In [1]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
import matplotlib.pyplot as plt
from shapely.geometry import mapping
sys.path.append(r'C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:\Users\mfpen\OneDrive\Documentos\Secretaría\yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\2136082468.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedo

In [2]:
# import warnings
# warnings.filterwarnings('ignore')
# import pandas as pd
# import math
# from shapely.geometry import Polygon
# import cv2
# from clasificacion_chinchetas import *
# import tqdm
# import numpy as np
# from osgeo import gdal
# import rasterio
# import geopandas as gpd
# import rasterio.mask
# import sys
# from shapely.geometry import mapping
# from shapely import geometry
# import cv2
# import numpy as np

## Raster y shape deteccion

In [3]:
raster=r"C:\Users\mfpen\Pictures\Ixtapan_Sal.tif"
src=rasterio.open(raster)

deteccion=r"D:\Secretaría\prueba_rotacion\capa_ixtapan_rot.shp"
deteccion_shp=gpd.read_file(deteccion)


In [4]:
deteccion_shp=deteccion_shp.dropna().reset_index(drop=True)
angulo_manzana=[]
for manzana in range(len(deteccion_shp)):
    proyecciones1=mapping(deteccion_shp['geometry'][manzana]).get('coordinates')
    angulos=[]
    d=[]
    poly=pd.DataFrame(proyecciones1[0])
    for point in range(1,len(poly)):
        d.append(((poly[1][point]-poly[1][point-1])**2+(poly[0][point]-poly[0][point-1])**2))
#         angulos.append(math.atan(((poly[1][point]-poly[1][point-1])/(poly[0][point]-poly[0][point-1])))*180/math.pi)
#     angulo_manzana.append(angulos[d.index(max(d))])
# deteccion_shp['angulo']=angulo_manzana
deteccion_shp1=ampliar_shape(deteccion_shp,factor_ampliacion=1.5)

## Clasificar casas y/o terrenos (Alexnet)

##### Pesos y diccionario Alexnet

In [5]:
# weights=r'C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint/best_mod11.pth'
# num_classes=5
# diciconario={0: 'casas', 1: 'en_construccion', 2: 'establecimiento', 3: 'multivivienda', 4: 'terreno_baldio'}
# model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)


# weights=r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint\best_modell11.pth"
# num_classes=6
# diciconario={0: 'carros', 1: 'casas', 2: 'en_construccion', 3: 'establecimiento', 4: 'multivivienda', 5: 'terreno_baldio'}
# model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)

#### Clasificación

In [6]:
def normalizar_array(img):
    return (img-np.min(img))/(np.max(img)-np.min(img))

In [7]:
%matplotlib
import tqdm
import cv2
geometry=[]
status=[]
for i,polygo in tqdm.tqdm(enumerate(deteccion_shp1["geometry"]),total=len(deteccion_shp1)):
    det="a"
    try:
        shapes=[mapping(polygo)]
    except:
        continue
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    imagen=cv2.cvtColor(np.moveaxis(array,0,-1),cv2.COLOR_RGB2BGR)
    imagen=np.array(imagen,dtype='int16')
    img=np.zeros_like(imagen)
    # img=np.array(img,"float")
    # for channel in range(img.shape[2]):
    for y in range(1,img.shape[0]-1):
        for x in range(1,img.shape[1]-1):
            for w in range(-1,2):
                for h in range(-1,2):
                    if np.min(np.abs(imagen[y+h,x+w,:]-imagen[y,x,:]))<=15:
                        img[y+h,x+w,:]=imagen[y,x,:]
                       # print(imagen[y,x,:])
    print(img.shape)
    
    maximo=np.max(img)
    minimo=np.min(img)
    cv2.imshow('prueba',img)
    cv2.waitKey()
    cv2.destroyAllWindows()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray=(normalizar_array(gray)*2-1)*255
    # plt.imshow(gray)
    # break
    y=gray.copy()
    y[y<5]=0
    y[y>=5]=255
    gray=cv2.erode(y,(6,6))
    # plt.imshow(gray)
    # break
    # cv2_imshow("y",np.array(np.hstack([gray,imagen[:,:,0]]),dtype="uint8"))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=np.array(img[:,:,1],dtype=float)**2
    h,w= img.shape
    yc,xc=int(h/2),int(w/2)
    thres=img.copy()
    u=190
    thres[thres<u**2]=0
    thres[thres>=u**2]=255
    thres=np.array(thres,dtype="uint8")#_,thres=cv2.threshold(img,173,255,1)
    f=1.25
    kernel = np.array([[-1,-1,1],[-1,1,-1],[1,-1,-1]])
    kernel=np.array([[1,1,1],[0,0,1],[0,0,1]])
    dst = cv2.filter2D(thres,-1,kernel)
    cont,hist=cv2.findContours(dst,2,1)
    res=pd.DataFrame({"c":cont,"area":[cv2.contourArea(cnt) for cnt in cont]})
    res.sort_values(by="area",ascending=False,inplace=True)
    res=res.reset_index(drop=True)
    try:
        cnt=res.c.values[0]
    except:
        geometry.append(polygo)
        status.append("error")
        continue
    if res.loc[0,"area"]<.5*polygo.area:
        geometry.append(polygo)
        status.append("area_small")
        continue
    epsilon = 0.02*cv2.arcLength(cnt,True)
    approx = cv2.approxPolyDP(cnt,epsilon,True)
    cnt=approx
    s=np.zeros_like(dst)
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    coords=shapes[0].get("coordinates")[0]
    left,bottom=np.min(coords,axis=0)
    right,top=np.max(coords,axis=0)
    height=imagen.shape[0]
    width=imagen.shape[1]
    # cv2.drawContours(s,[box],0,(127),2)
    # cv2.drawContours(s,[cnt],-1,(255),-1)
    geometry.append(Polygon([(map_d(x[0],0,width,left,right),map_d(x[1],height,0,bottom,top)) for x in box]))
    status.append("correcto")
    # cv2_imshow("s",s)
    # plt.show()
    #break
    # gray = np.float32(gray)
    # dst = cv.cornerHarris(gray,2,3,0.04)
    # dst[dst<=0]=0
    # dst=dst
    # sa=dst.copy()
    # sa=sa.reshape(-1)
    # sa.sort()
    # box=np.moveaxis(np.where(dst>=sa[-4]),0,-1)
    # # print(box)
    # box=np.stack([box[:,1],box[:,0]],axis=-1)
    # # box
    # # cv2.drawContours(img,[box],0,(0,0,255),2)
    # cv2_imshow("s",img)
#     rows,cols = img.shape[:2]
#     [vx,vy,x,y] = cv.fitLine(box, cv.DIST_L2,0,0.01,0.01)
#     lefty = int((-x*vy/vx) + y)
#     righty = int(((cols-x)*vy/vx)+y)
#     cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)
#     cv2_imshow("s",img)
#     # four_images=[array[2],array[1],array[0]]
#     # stacked_images = np.stack(four_images, axis=-1)
    
# #     imagen_1=rotate_image(stacked_images,deteccion_shp["angulo"][i],reshape=False)
# #     clase,imagen=model_class.predict_image(imagen_1)
    
# #     cv2.|imshow("im",imagen)
# #     cv2.waitKey()
# #     cv2.destroyAllWindows()
#     deteccion_shp.loc[i,"clase_n"]=clase

Using matplotlib backend: <object object at 0x0000028752BBCD80>


  0%|          | 0/333 [00:00<?, ?it/s]

(26, 44, 3)


  0%|          | 0/333 [00:08<?, ?it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\453727112.py", line 33, in <module>
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.error: OpenCV(4.7.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x981fb336::Set<1,-1,-1>,struct cv::impl::A0x981fb336::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 3 (CV_16S)


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\m

In [ ]:
ñ

In [8]:
type(array)

numpy.ndarray

In [9]:
(np.array([1.1,2.1,3.1,4.1],dtype='float32'))

array([        1.1,         2.1,         3.1,         4.1], dtype=float32)

In [10]:
gpd.GeoDataFrame({"status":status},geometry=geometry,crs=3857).to_file(r"D:\Secretaría\prueba_rotacion/test_nuevarotacion.shp")


In [11]:
res.loc[i,"c"]

#  box_l.append(Polygon([(map_d(x[0],0,width,width_min,width_max),map_d(x[1],heigth,0,heigth_min,height_max)) for x in box]))
coords=shapes[0].get("coordinates")[0]
left,bottom=np.min(coords,axis=0)
right,top=np.max(coords,axis=0)


height=imagen.shape[0]
width=imagen.shape[1]
left,bottom,right,top,height,width


           

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\2764675335.py", line 1, in <module>
    res.loc[i,"c"]
NameError: name 'res' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1177, in structured_traceback
    return VerboseTB.structured_traceback(
  File "c:\Users\mfpen\anaconda

In [12]:
geomtry=(Polygon([(map_d(x[0],0,width,left,right),map_d(x[1],height,0,bottom,top)) for x in box]))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\3263067697.py", line 1, in <module>
    geomtry=(Polygon([(map_d(x[0],0,width,left,right),map_d(x[1],height,0,bottom,top)) for x in box]))
NameError: name 'box' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1177, in structured_

In [13]:
deteccion_shp

,clase_dete,conf,area,index_righ,Clase,geometry,centroid
0,establecimiento,0.69798,112.630907,652.0,0|0|0|0,"POLYGON ((-11096049.325 2136857.093, -11096035...",POINT (-11096042.238 2136853.203)
1,casas,0.41600,120.675972,660.0,0|0|0|0,"POLYGON ((-11096039.934 2136689.909, -11096033...",POINT (-11096037.278 2136679.779)
2,establecimiento,0.47930,96.540777,656.0,tiendas_1|tiendas_1|tiendas_1|tiendas_1,"POLYGON ((-11096040.818 2136722.060, -11096034...",POINT (-11096038.092 2136713.935)
3,casas,0.23450,135.760468,725.0,hospedaje_google|hospedaje_google|hospedaje_go...,"POLYGON ((-11096009.695 2137351.480, -11095994...",POINT (-11096001.880 2137347.502)
4,establecimiento,0.42260,199.115353,725.0,hospedaje_google|hospedaje_google|hospedaje_go...,"POLYGON ((-11096007.206 2137344.615, -11095989...",POINT (-11095997.821 2137339.741)
...,...,...,...,...,...,...,...
328,establecimiento,0.51276,286.605432,1439.0,0|0|0|0,"POLYGON ((-11094722.135 2137349.280, -11094706...",POINT (-11094718.444 2137337.714)
329,establecimiento,0.52256,286.605432,1439.0,0|0|0|0,"POLYGON ((-11094742.121 2137355.510, -11094729...",POINT (-11094741.213 2137343.402)
330,casas,0.68588,307.723727,1439.0,0|0|0|0,"POLYGON ((-11094736.195 2137371.475, -11094722...",POINT (-11094734.176 2137359.223)
331,establecimiento,0.78382,418.343368,1437.0,a_verdes|a_verdes|a_verdes|a_verdes,"POLYGON ((-11094713.958 2137370.381, -11094692...",POINT (-11094707.638 2137356.436)


In [14]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
# img=cv2.imread("Alarms/malavi/industria~2.png")
# img=cv2.resize(img,(60,60))
# img=cv2.resize(img,(100,100))
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img=np.array(img[:,:,1],dtype=float)**2
h,w= img.shape
yc,xc=int(h/2),int(w/2)
thres=img.copy()
u=190
thres[thres<u**2]=0
thres[thres>=u**2]=255
thres=np.array(thres,dtype="uint8")#_,thres=cv2.threshold(img,173,255,1)
f=1.25
kernel = np.array([[-1,-1,1],[-1,1,-1],[1,-1,-1]])
kernel=np.array([[1,1,1],[0,0,1],[0,0,1]])
dst = cv2.filter2D(thres,-1,kernel)
cont,hist=cv2.findContours(dst,2,1)
res=pd.DataFrame({"c":cont,"s":[cv2.contourArea(cnt) for cnt in cont]})
res.sort_values(by="s",ascending=False,inplace=True)
cnt=res.c.values[0]
epsilon = 0.02*cv2.arcLength(cnt,True)
approx = cv2.approxPolyDP(cnt,epsilon,True)
cnt=approx
s=np.zeros_like(dst)
rect = cv2.minAreaRect(cnt)
box = cv2.boxPoints(rect)
box = np.int0(box)
cv2.drawContours(s,[box],0,(127),2)
cv2.drawContours(s,[cnt],-1,(255),-1)
plt.imshow(s)
plt.show()
print(cont)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\1970685654.py", line 8, in <module>
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
cv2.error: OpenCV(4.7.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x981fb336::Set<3,4,-1>,struct cv::impl::A0x981fb336::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 3 (CV_16S)


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mfpe

In [15]:
y=gray.copy()
y[y<200]=0
y=cv2.erode(y,(6,6))
cv2_imshow("y",y)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\2762985429.py", line 1, in <module>
    y=gray.copy()
NameError: name 'gray' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1177, in structured_traceback
    return VerboseTB.structured_traceback(
  File "c:\Users\mfpen\anaconda

In [16]:
pd.DataFrame(gray).to_clipboard()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\2500834726.py", line 1, in <module>
    pd.DataFrame(gray).to_clipboard()
NameError: name 'gray' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1177, in structured_traceback
    return VerboseTB.structured_traceback(
  File "c:\

In [17]:
import numpy as np
import cv2 as cv
filename = 'chessboard.png'
img =imagen.copy()
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
gray = np.float32(gray)
dst = cv.cornerHarris(gray,2,3,0.04)
dst[dst<=0]=0
dst=dst**2
sa=dst.copy()
sa=sa.reshape(-1)
sa.sort()
box=np.moveaxis(np.where(dst>=sa[-4]),0,-1)
print(box)
box=np.stack([box[:,1],box[:,0]],axis=-1)
# box
# cv2.drawContours(img,[box],0,(0,0,255),2)
# cv2_imshow("s",img)
rows,cols = img.shape[:2]
[vx,vy,x,y] = cv.fitLine(box, cv.DIST_L2,0,0.01,0.01)
lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)
cv2_imshow("s",img)
#result is dilated for marking the corners, not important
# dst = cv.dilate(dst,None)
# Threshold for an optimal value, it may vary depending on the image.
# img[dst>0.01*dst.max()]=
img[dst>0.05*dst.max()]=[0,0,255]
cv2_imshow('dst',img)
# if cv.waitKey(0) & 0xff == 27:
#     cv.destroyAllWindows()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\986693823.py", line 5, in <module>
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
cv2.error: OpenCV(4.7.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x981fb336::Set<1,-1,-1>,struct cv::impl::A0x981fb336::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 3 (CV_16S)


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mfpe

In [18]:
%matplotlib

xp, yp,_= img.shape

x = np.arange(0, xp, 1)
y = np.arange(0, yp, 1)
Y, X = np.meshgrid(y, x)
fig=plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(X,Y,dst,c=np.random.randint(0,255,len(dst.reshape(-1)))/255)

Using matplotlib backend: QtAgg
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\4031011159.py", line 10, in <module>
    ax.scatter(X,Y,dst,c=np.random.randint(0,255,len(dst.reshape(-1)))/255)
NameError: name 'dst' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1177, in structured_traceback
    return Verbo

In [19]:
sa=dst.copy()
sa=sa.reshape(-1)
sa.sort()
box=np.moveaxis(np.where(dst>=sa[-4]),0,-1)
print(box)
box=np.stack([box[:,1],box[:,0]],axis=-1)
# box
# cv2.drawContours(img,[box],0,(0,0,255),2)
# cv2_imshow("s",img)
rows,cols = img.shape[:2]
[vx,vy,x,y] = cv.fitLine(box, cv.DIST_L2,0,0.01,0.01)
lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)
cv2_imshow("s",img)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\554323224.py", line 1, in <module>
    sa=dst.copy()
NameError: name 'dst' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1177, in structured_traceback
    return VerboseTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\

## Clasificar chinchetas (Matching template)

#### Plantillas

In [20]:
# No borrar path compañeros

path_ruben = r'C:\Users\ruben\Desktop\GEM\Geo\Bases\Plantillas'
path_hector = '/home/hector/Documentos/Infis/Geo/Data/Plantillas'
dict_gris = get_dict_plantilla_gris(path_ruben,True,(30,30))

#### Clasificación

In [21]:
import tqdm
import cv2
dict_gris = get_dict_plantilla_gris(path_ruben,True,(30,30))
for i,polygo in tqdm.tqdm(enumerate(deteccion_shp1["geometry"]),total=len(deteccion_shp1)):
    det="a"
    try:
        shapes=[mapping(polygo)]
    except:
        continue
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    four_images=[array[2],array[1],array[0]]
    stacked_images = np.stack(four_images, axis=-1)
    
    imagen_1=rotate_image(stacked_images,deteccion_shp["angulo"][i],reshape=False)

    # clasificacion
    clase_aux, n_aux, umbral_aux  = iter_umbral_fn(imagen_1, dict_gris,n=100,salto_n=10,
                                                    umbral=0.94,min_umbral=0.8)
    
    # Revisamos la mayor confusion que se da ... areas_verdes ~ establecimiento_google
    if clase_aux in ['a_verdes','establecimiento_google']:
        clase_aux = areasv_vs_estabgoogle(imagen_1, dict_gris)    
  
    deteccion_shp.loc[i,"clase_chincheta"] = clase_aux
    deteccion_shp.loc[i,"umbral"] = umbral_aux
    deteccion_shp.loc[i,"n_resize"] = n_aux

  0%|          | 0/333 [00:00<?, ?it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\mfpen\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexes\base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'angulo'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_9964\1719119686.py", line 15, in <module>
    imagen